In [3]:
%logstop
%logstart -rtq ~/.logs/dw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [4]:
from static_grader import grader

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We'll work with the same medical data used in the `pw` miniproject but leverage the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [5]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

mkdir: cannot create directory ‘dw-data’: File exists
File ‘./dw-data/201701scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/201606scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/practices.csv.gz’ already there; not retrieving.

File ‘./dw-data/chem.csv.gz’ already there; not retrieving.



## Loading the data

Similar to the `PW` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a Pandas DataFrame by making use of the `gzip` package to decompress the files and Panda's `read_csv` methods to parse the data into a DataFrame. You may want to check the Pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [PW miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [6]:
import pandas as pd
import numpy as np
import gzip

In [7]:
# load the 2017 data
scripts = pd.read_csv('dw-data/201701scripts_sample.csv.gz')
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [8]:
scripts.shape

(973193, 7)

In [9]:
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
practices = pd.read_csv('dw-data/practices.csv.gz', names=col_names)
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [10]:
chem = pd.read_csv('dw-data/chem.csv.gz')
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


Now that we've loaded in the data, let's first replicate our results from the `PW` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `PW` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `PW` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: [('act_cost', (total, mean, std, q25, median, q75)), ...]

In [11]:
#summary_stats = [('items', (0,) * 6), ('quantity', (0,) * 6), ('nic', (0,) * 6), ('act_cost', (0,) * 6)]

In [12]:
desc = scripts.describe()
stats = ['mean', 'std', '25%', '50%', '75%']
summary_stats = []
for col in ['items', 'quantity', 'nic', 'act_cost']:
    summary_stats.append((col, tuple([scripts[col].sum()] + [desc.loc[st, col] for st in stats])))

In [13]:
grader.score.dw__summary_statistics(summary_stats)

Your score:  1.0


## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [14]:
#most_common_item = [("", 0)]

In [15]:
sorted_items = scripts.groupby('bnf_name').sum()['items'].sort_values()
most_common_item = list(dict(sorted_items.tail(1)).items())

In [16]:
grader.score.dw__most_common_item(most_common_item)

Your score:  1.0


## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the PW miniproject).

In [17]:
script_copy = scripts.copy()
practice_copy = practices.copy()[['code', 'post_code']]
script_copy.rename(columns={'practice': 'prac_code'}, inplace=True)
practice_copy.rename(columns={'code': 'prac_code'}, inplace=True)
script_copy.head()

,prac_code,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [18]:
practice_copy.head()

,prac_code,post_code
0,A81001,TS18 1HU
1,A81002,TS18 2AW
2,A81003,TS26 8DB
3,A81004,TS1 3BE
4,A81005,TS14 7DJ


In [19]:
practice_copy = practice_copy.groupby('prac_code').apply(lambda x: x.sort_values(by='post_code').iloc[0])

In [20]:
practice_copy = practice_copy.drop('prac_code', axis=1).reset_index()

In [21]:
joined = pd.merge(script_copy, practice_copy, how='left', on='prac_code')
joined.head()

,prac_code,bnf_code,bnf_name,items,nic,act_cost,quantity,post_code
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,CH44 5UF
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,CH44 5UF
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,CH44 5UF
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,CH44 5UF
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,CH44 5UF


In [22]:
assert joined.shape[0] == scripts.shape[0]

In [23]:
def get_max_item(mini_df):
    mini_df = mini_df.groupby('bnf_name').sum().reset_index()
    mini_df['frac_item'] = mini_df['items'] / mini_df['items'].sum()
    return mini_df.sort_values(by='frac_item').iloc[-1]

In [24]:
items_by_region =joined.groupby('post_code').apply(get_max_item)
items_by_region.head()

,bnf_name,items,nic,act_cost,quantity,frac_item
post_code,,,,,,
B11 4BW,Salbutamol_Inha 100mcg (200 D) CFF,706,1623.00,1511.26,1082,0.031059
B12 9LP,Paracet_Tab 500mg,425,996.10,931.32,45490,0.024893
B18 7AL,Salbutamol_Inha 100mcg (200 D) CFF,556,1014.00,945.62,676,0.027111
B21 9RY,Metformin HCl_Tab 500mg,1033,3849.19,3589.00,125316,0.033294
B23 6DJ,Lansoprazole_Cap 30mg (E/C Gran),599,891.83,841.15,22445,0.021384


In [25]:
#items_by_region = [("B11 4BW", "Salbutamol_Inha 100mcg (200 D) CFF", 0.0310589063)] * 100

In [26]:
items_by_region = [(p, items_by_region.loc[p, 'bnf_name'], items_by_region.loc[p, 'frac_item']) \
                   for p in items_by_region.index[:100]]

In [27]:
grader.score.dw__items_by_region(items_by_region)

Your score:  1.0


## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [28]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table.. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

In [29]:
chem_copy = chem.copy()
chem_copy['is_opioids'] = [int(i) for i in chem_copy['NAME'].str.contains('|'.join(opioids), case=False)]
chem_copy.head()

,CHEM SUB,NAME,is_opioids
0,0101010A0,Alexitol Sodium,0
1,0101010B0,Almasilate,0
2,0101010C0,Aluminium Hydroxide,0
3,0101010D0,Aluminium Hydroxide With Magnesium,0
4,0101010E0,Hydrotalcite,0


In [30]:
is_opioids_df = chem_copy.groupby('CHEM SUB').first()['is_opioids'].reset_index()
is_opioids_df = is_opioids_df.rename(columns={0: 'is_opioids', 'CHEM SUB': 'bnf_code'})
is_opioids_df.head()

,bnf_code,is_opioids
0,010101000,0
1,0101010A0,0
2,0101010B0,0
3,0101010C0,0
4,0101010D0,0


In [31]:
joined = pd.merge(joined, is_opioids_df, on='bnf_code', how='left')
print(joined.shape)
joined.head()

(973193, 9)


,prac_code,bnf_code,bnf_name,items,nic,act_cost,quantity,post_code,is_opioids
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,CH44 5UF,0.0
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,CH44 5UF,0.0
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,CH44 5UF,0.0
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,CH44 5UF,0.0
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,CH44 5UF,0.0


In [32]:
joined.isnull().sum()

prac_code          0
bnf_code           0
bnf_name           0
items              0
nic                0
act_cost           0
quantity           0
post_code          0
is_opioids    118749
dtype: int64

In [33]:
joined.fillna(0, inplace=True)
joined.isnull().sum()

prac_code     0
bnf_code      0
bnf_name      0
items         0
nic           0
act_cost      0
quantity      0
post_code     0
is_opioids    0
dtype: int64

Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

In [34]:
opioids_per_practice = joined.groupby('prac_code').mean()['is_opioids']
opioids_per_practice.head()

prac_code
A81005    0.033179
A81007    0.043329
A81011    0.046556
A81012    0.042793
A81017    0.038140
Name: is_opioids, dtype: float64

How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

In [35]:
relative_opioids_per_practice = opioids_per_practice - joined['is_opioids'].mean()
relative_opioids_per_practice.head()

prac_code
A81005   -0.002624
A81007    0.007526
A81011    0.010753
A81012    0.006990
A81017    0.002337
Name: is_opioids, dtype: float64

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [36]:
n_scripts = joined.groupby('prac_code').count()['is_opioids']
n_scripts.name = 'n_script'
n_scripts.head()

prac_code
A81005    1507
A81007    1454
A81011    1568
A81012    1332
A81017    2150
Name: n_script, dtype: int64

In [37]:
#relative_opioids_per_practice = relative_opioids_per_practice.reset_index()
#relative_opioids_per_practice.head()

In [38]:
standard_error_per_practice = np.std(joined['is_opioids']) / np.sqrt(n_scripts)
standard_error_per_practice.head()

prac_code
A81005    0.004786
A81007    0.004873
A81011    0.004692
A81012    0.005091
A81017    0.004007
Name: n_script, dtype: float64

In [39]:
opioid_scores = relative_opioids_per_practice / standard_error_per_practice
opioid_scores.head()

prac_code
A81005   -0.548306
A81007    1.544558
A81011    2.291796
A81012    1.373061
A81017    0.583168
dtype: float64

The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [40]:
opioid_scores = opioid_scores.reset_index()
opioid_scores.head()

,prac_code,0
0,A81005,-0.548306
1,A81007,1.544558
2,A81011,2.291796
3,A81012,1.373061
4,A81017,0.583168


In [41]:
practice_copy2 = practices.copy()[['code', 'name']]
practice_copy2 = practice_copy2.rename(columns={'code': 'prac_code'})
practice_copy2 = practice_copy2.groupby('prac_code').apply(lambda x: x.sort_values(by='name').iloc[0])
practice_copy2.head()

,prac_code,name
prac_code,,
A81001,A81001,THE DENSHAM SURGERY
A81002,A81002,QUEENS PARK MEDICAL CENTRE
A81003,A81003,VICTORIA MEDICAL PRACTICE
A81004,A81004,BLUEBELL MEDICAL CENTRE
A81005,A81005,SPRINGWOOD SURGERY


In [42]:
practice_copy2 = practice_copy2.drop('prac_code', axis=1).reset_index()
practice_copy2.head()

,prac_code,name
0,A81001,THE DENSHAM SURGERY
1,A81002,QUEENS PARK MEDICAL CENTRE
2,A81003,VICTORIA MEDICAL PRACTICE
3,A81004,BLUEBELL MEDICAL CENTRE
4,A81005,SPRINGWOOD SURGERY


In [43]:
n_scripts.reset_index().head()

,prac_code,n_script
0,A81005,1507
1,A81007,1454
2,A81011,1568
3,A81012,1332
4,A81017,2150


In [44]:
opioid_scores = pd.merge(opioid_scores, practice_copy2, how='left', on='prac_code')
opioid_scores = pd.merge(opioid_scores, n_scripts.reset_index(), how='left', on='prac_code')
opioid_scores = opioid_scores.rename(columns={0: 'z_score'})
opioid_scores.head()

,prac_code,z_score,name,n_script
0,A81005,-0.548306,SPRINGWOOD SURGERY,1507
1,A81007,1.544558,BANKHOUSE SURGERY,1454
2,A81011,2.291796,CHADWICK PRACTICE,1568
3,A81012,1.373061,WESTBOURNE MEDICAL CENTRE,1332
4,A81017,0.583168,WOODBRIDGE PRACTICE,2150


In [45]:
#unique_practices = ...
#anomalies = [("NATIONAL ENHANCED SERVICE", 11.6958178629, 7)] * 100

In [46]:
results = opioid_scores.sort_values(by='z_score', ascending=False).iloc[:100, 1:]
results.head()

,z_score,name,n_script
714,11.695824,NATIONAL ENHANCED SERVICE,7
754,7.339047,OUTREACH SERVICE NH / RH,2
772,6.150585,BRISDOC HEALTHCARE SERVICES OOH,60
261,5.123035,H&R P C SPECIAL SCHEME,36
819,4.958869,HMR BARDOC OOH,321


In [47]:
opioid_scores.isnull().sum()

prac_code    0
z_score      0
name         0
n_script     0
dtype: int64

In [48]:
anomalies = [(results.iloc[row, 1], results.iloc[row, 0], results.iloc[row, 2]) for row in range(results.shape[0])]
anomalies[:5]

[('NATIONAL ENHANCED SERVICE', 11.695823871932637, 7),
 ('OUTREACH SERVICE NH / RH', 7.339046789841713, 2),
 ('BRISDOC HEALTHCARE SERVICES OOH', 6.150584909087287, 60),
 ('H&R P C SPECIAL SCHEME', 5.123035046109328, 36),
 ('HMR BARDOC OOH', 4.958868986219792, 321)]

In [49]:
grader.score.dw__script_anomalies(anomalies)

Your score:  1.0


## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the percent growth in prescription rate from June 2016 to January 2017 for each `bnf_name`. We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative percent growth) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [50]:
scripts16 = pd.read_csv('dw-data/201606scripts_sample.csv.gz')
scripts16.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85638,0301011R0,Salamol_Inha 100mcg (200 D) CFF (Teva),2,2.92,2.73,2
1,N85638,0301011R0,Easyhaler_Salbutamol Sulf 200mcg (200D),1,6.63,6.15,1
2,N85638,0301020I0,Ipratrop Brom_Inh Soln 500mcg/2ml Ud,1,1.77,1.75,12
3,N85638,0301020I0,Ipratrop Brom_Inh Soln 250mcg/1ml Ud,1,4.47,4.15,20
4,N85638,0302000C0,Clenil Modulite_Inha 50mcg (200D),1,3.70,3.44,1


In [51]:
scripts16_copy = scripts16.copy()
bnf_rate16 = scripts16_copy.groupby('bnf_name').count()['items']
bnf_rate17 = joined.groupby('bnf_name').count()['items']

In [52]:
bnf_rate16 = bnf_rate16[bnf_rate16 >= 50]

In [53]:
bnf_16_17_df = pd.merge(bnf_rate16.reset_index(), bnf_rate17.reset_index(), how='left', on='bnf_name')
bnf_16_17_df.head()

,bnf_name,items_x,items_y
0,3m Health Care_Cavilon Durable Barrier C,825,816.0
1,3m Health Care_Cavilon No Sting 1ml Barr,231,223.0
2,3m Health Care_Cavilon No Sting 3ml Barr,105,94.0
3,3m Health Care_Cavilon No Sting Barrier,454,412.0
4,A.S Saliva Orthana Spy 50ml (App),84,121.0


In [54]:
bnf_16_17_df.isnull().sum()

bnf_name     0
items_x      0
items_y     34
dtype: int64

In [55]:
bnf_16_17_df.dropna(axis=0, inplace=True)
bnf_16_17_df.isnull().sum()

bnf_name    0
items_x     0
items_y     0
dtype: int64

In [56]:
bnf_16_17_df['diff_rate'] = (bnf_16_17_df['items_y'] - bnf_16_17_df['items_x']) / bnf_16_17_df['items_x']
bnf_16_17_df.head()

,bnf_name,items_x,items_y,diff_rate
0,3m Health Care_Cavilon Durable Barrier C,825,816.0,-0.010909
1,3m Health Care_Cavilon No Sting 1ml Barr,231,223.0,-0.034632
2,3m Health Care_Cavilon No Sting 3ml Barr,105,94.0,-0.104762
3,3m Health Care_Cavilon No Sting Barrier,454,412.0,-0.092511
4,A.S Saliva Orthana Spy 50ml (App),84,121.0,0.440476


In [57]:
bnf_16_17_df.isnull().sum()

bnf_name     0
items_x      0
items_y      0
diff_rate    0
dtype: int64

In [58]:
bnf_16_17_df.sort_values(by='diff_rate', ascending=False, inplace=True)

In [59]:
bnf_16_17_df.head()

,bnf_name,items_x,items_y,diff_rate
433,Butec_Transdermal Patch 5mcg/hr,62,277.0,3.467742
432,Butec_Transdermal Patch 10mcg/hr,69,276.0,3.000000
1419,Fostair NEXThaler_Inh 200mcg/6mcg (120D),86,209.0,1.430233
2636,Pneumococcal_Vac 0.5ml Vl (23 Valent),193,438.0,1.269430
3063,Spiolto Respimat_Inha2.5/2.5mcg(60D)+Dev,52,118.0,1.269231


In [60]:
top_50 = bnf_16_17_df.iloc[:50, [0, 1, 3]]
bottom_50 = bnf_16_17_df.iloc[-50:, [0, 1, 3]]

In [61]:
top_bottom = pd.concat([top_50, bottom_50])
top_bottom.shape

(100, 3)

In [62]:
top_bottom.isnull().sum()

bnf_name     0
items_x      0
diff_rate    0
dtype: int64

In [63]:
script_growth = [(top_bottom['bnf_name'].iloc[i], top_bottom['diff_rate'].iloc[i], top_bottom['items_x'].iloc[i]) for i in \
                 range(top_bottom.shape[0])]

In [64]:
script_growth[:5]

[('Butec_Transdermal Patch 5mcg/hr', 3.467741935483871, 62),
 ('Butec_Transdermal Patch 10mcg/hr', 3.0, 69),
 ('Fostair NEXThaler_Inh 200mcg/6mcg (120D)', 1.430232558139535, 86),
 ('Pneumococcal_Vac 0.5ml Vl (23 Valent)', 1.2694300518134716, 193),
 ('Spiolto Respimat_Inha2.5/2.5mcg(60D)+Dev', 1.2692307692307692, 52)]

In [65]:
#script_growth = [("Butec_Transdermal Patch 5mcg\/hr", 3.4677419355, 62.0)] * 100

In [66]:
grader.score.dw__script_growth(script_growth)

Your score:  1.0


## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [67]:
p = 1 / len(joined['bnf_code'].unique())
grp_by_bnf_code = joined.groupby('bnf_code').count()[['items']]
grp_by_bnf_code.reset_index(inplace=True)
grp_by_bnf_code['rates'] = grp_by_bnf_code['items'] / grp_by_bnf_code['items'].sum()

rare_codes = grp_by_bnf_code[grp_by_bnf_code['rates'] < 0.1*p]['bnf_code']

joined['is_rare'] = [float(i) for i in joined['bnf_code'].isin(rare_codes)]
joined.head()

,prac_code,bnf_code,bnf_name,items,nic,act_cost,quantity,post_code,is_opioids,is_rare
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,CH44 5UF,0.0,0.0
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,CH44 5UF,0.0,0.0
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,CH44 5UF,0.0,0.0
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,CH44 5UF,0.0,0.0
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,CH44 5UF,0.0,0.0


Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [68]:
def get_rare_cost(mini_df):
    rare_cost = mini_df[mini_df['is_rare'] == 1]['act_cost'].sum()
    df = pd.DataFrame({'rare_cost_sum': [rare_cost], 'act_cost_sum': [mini_df['act_cost'].sum()]})
    return df.iloc[0]

In [69]:
rare_cost_prop = joined.groupby('prac_code').apply(get_rare_cost)

In [70]:
rare_cost_prop['rare_cost_prop'] = rare_cost_prop['rare_cost_sum'] / rare_cost_prop['act_cost_sum']
rare_cost_prop.head()

,rare_cost_sum,act_cost_sum,rare_cost_prop
prac_code,,,
A81005,1247.83,103840.82,0.012017
A81007,951.06,113482.49,0.008381
A81011,816.02,159507.03,0.005116
A81012,1145.11,83296.81,0.013747
A81017,1712.15,232656.17,0.007359


Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [71]:
relative_rare_cost_prop = rare_cost_prop['rare_cost_prop'] - \
(joined[joined['is_rare'] == 1]['act_cost'].sum() / joined['act_cost'].sum())

relative_rare_cost_prop.head()

prac_code
A81005   -0.003946
A81007   -0.007582
A81011   -0.010847
A81012   -0.002216
A81017   -0.008604
Name: rare_cost_prop, dtype: float64

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [72]:
standard_errors = np.std(relative_rare_cost_prop)
standard_errors

0.060473532762903806

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(post_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [73]:
rare_scores = relative_rare_cost_prop / standard_errors

In [74]:
rare_scores.name = 'rare_score'
rare_scores.reset_index().head()

,prac_code,rare_score
0,A81005,-0.065254
1,A81007,-0.125381
2,A81011,-0.179368
3,A81012,-0.036637
4,A81017,-0.142273


In [75]:
practice_copy3 = practices.copy()
practice_copy3.rename(columns={'code': 'prac_code'}, inplace=True)

In [76]:
practice_copy3 = practice_copy3.groupby('prac_code').apply(lambda x: x.sort_values(by='name').iloc[0])

In [77]:
practice_copy3.drop('prac_code', axis=1, inplace=True)
practice_copy3.reset_index(inplace=True)
practice_copy3.head()

,prac_code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,BLUEBELL MEDICAL CENTRE,TRIMDON AVENUE,ACKLAM,MIDDLESBROUGH,NaN,TS5 8SB
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [78]:
rare_scripts_df = pd.merge(practice_copy3, rare_scores.reset_index(), how='left', on='prac_code')
rare_scripts_df.sort_values(by='rare_score', ascending=False, inplace=True)
rare_scripts_df.head()

,prac_code,name,addr_1,addr_2,borough,village,post_code,rare_score
8948,Y03472,CONSULTANT DIABETES TEAM,HEALTHY LIVING CENTRE,PRINCES STREET,PETERBOROUGH,CAMBRIDGESHIRE,PE1 2QP,16.272195
10508,Y05320,DMC COMMUNITY DERMATOLOGY RBWF,BHI PARKSIDE,STOURBRIDGE ROAD,BROMSGROVE,NaN,B61 0AZ,15.137493
9702,Y04404,OUTPATIENTS JUBILEE HEALTH CENTRE,JUBILEE HEALTH CENTRE,SHOTFIELD,WALLINGTON,SURREY,SM6 0HY,7.546549
8955,Y03484,DMC COMMUNITY DERMATOLOGY CLINIC,SHEPPEY COMMUNITY HOSP,PLOVER ROAD,MINSTER ON SEA SHEERNESS,KENT,ME12 3LT,7.291482
9721,Y04424,DMC HEALTHCARE,CANVEY ISLAND P.C.C.,LONG ROAD,CANVEY ISLAND,ESSEX,SS8 0JA,6.842612


In [79]:
#rare_scripts = [("Y03472", "CONSULTANT DIABETES TEAM", 16.2626871247)] * 100

In [80]:
rare_scripts = [(rare_scripts_df['prac_code'].iloc[i], rare_scripts_df['name'].iloc[i], rare_scripts_df['rare_score'].iloc[i]) \
                for i in range(100)]

In [81]:
rare_scripts[:5]

[('Y03472', 'CONSULTANT DIABETES TEAM', 16.27219468882569),
 ('Y05320', 'DMC COMMUNITY DERMATOLOGY RBWF', 15.137492772726269),
 ('Y04404', 'OUTPATIENTS JUBILEE HEALTH CENTRE', 7.546548675017432),
 ('Y03484', 'DMC COMMUNITY DERMATOLOGY CLINIC', 7.291482488414039),
 ('Y04424', 'DMC HEALTHCARE', 6.8426122023423055)]

In [82]:
grader.score.dw__rare_scripts(rare_scripts)

Your score:  1.0


*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*